# Preprocess of house prices dataset
In the following notebooks we're going to preprocess the data, that is remove missing variables, transform the variables and treat outliers. We're also going to build a specialized pipeline for those transformations.

In this notebook specifically, we're going to fill missing values in test data.

In [1]:
# import dataset and libraries
import numpy as np
import pandas as pd

In [2]:
orig_data = pd.read_csv("data/test.csv", index_col="Id")

In [3]:
# copying the dataset for analysis
house_data = orig_data.copy()
house_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


In [5]:
print(house_data['MiscFeature'][house_data["MiscVal"] == 0].isnull().sum())
print(house_data['PoolQC'][house_data["PoolArea"] == 0].isnull().sum())
print(house_data['FireplaceQu'][house_data["Fireplaces"] == 0].isnull().sum())

1407
1453
730


It seems there is an additional value in MiscFeature that needs attention, that doesn't have 0 miscellaneous value. Let's print it:

In [6]:
house_data[house_data["MiscVal"] != 0][house_data['MiscFeature'].isnull()]

<ipython-input-6-31f53ea17f2c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  house_data[house_data["MiscVal"] != 0][house_data['MiscFeature'].isnull()]


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
2550,20,RL,128.0,39290,Pave,NaN,IR1,Bnk,AllPub,Inside,...,0,0,NaN,NaN,NaN,17000,10,2007,New,Partial


It appears to be mistakenly assigned to lack of feature since its value is above 0. I will change the value for this feature to "Othr":

In [7]:
house_data.loc[2550, "MiscFeature"] = "Othr"

In [8]:
# As explained in missing values of train data, we can fill those features immediately:
house_data["PoolQC"] = house_data["PoolQC"].fillna("NoPool")
house_data["MiscFeature"] = house_data["MiscFeature"].fillna("NoFeature")
house_data["Fence"] = house_data["Fence"].fillna("NoFence")
house_data["Alley"] = house_data["Alley"].fillna("NoAlley")
house_data["FireplaceQu"] = house_data["FireplaceQu"].fillna("NoFireplace")

In [9]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSZoning          4
LotFrontage     227
Utilities         2
Exterior1st       1
Exterior2nd       1
MasVnrType       16
MasVnrArea       15
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinSF1        1
BsmtFinType2     42
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
KitchenQual       1
Functional        2
GarageType       76
GarageYrBlt      78
GarageFinish     78
GarageCars        1
GarageArea        1
GarageQual       78
GarageCond       78
SaleType          1
dtype: int64


From the rest we can easily set most of the missing values to modes:

In [10]:
mode_replacement = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "BsmtFullBath", "BsmtHalfBath", 
                    "KitchenQual", "Functional", "SaleType"]

for feature in mode_replacement:
    house_data[feature] = house_data[feature].fillna(house_data[feature].mode()[0])

In [11]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage     227
MasVnrType       16
MasVnrArea       15
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinSF1        1
BsmtFinType2     42
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
GarageType       76
GarageYrBlt      78
GarageFinish     78
GarageCars        1
GarageArea        1
GarageQual       78
GarageCond       78
dtype: int64


Let's first fill the missing values in "Mas" features:

In [12]:
house_data["MasVnrArea"][house_data["MasVnrType"].isnull()].isnull().sum()

15

In [13]:
house_data[house_data["MasVnrType"].isnull()][house_data["MasVnrArea"].notnull()][["MasVnrArea", "MasVnrType"]]

<ipython-input-13-5f1042505c6c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  house_data[house_data["MasVnrType"].isnull()][house_data["MasVnrArea"].notnull()][["MasVnrArea", "MasVnrType"]]


,MasVnrArea,MasVnrType
Id,,
2611,198.0,NaN


Since the area is known I cannot assign type to None type. I will instead fill it with the mode out of non-None values

In [14]:
house_data.loc[2611, "MasVnrType"] = house_data["MasVnrType"][house_data["MasVnrType"] != "None"].mode()[0]

In [15]:
# the rest will be filled with 0s
house_data["MasVnrArea"] = house_data["MasVnrArea"].fillna(0)
house_data["MasVnrType"] = house_data["MasVnrType"].fillna("None")

In [16]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage     227
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinSF1        1
BsmtFinType2     42
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
GarageType       76
GarageYrBlt      78
GarageFinish     78
GarageCars        1
GarageArea        1
GarageQual       78
GarageCond       78
dtype: int64


In [17]:
for feature in ["GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond"]:
    print(house_data[feature][house_data["GarageType"].isnull()].isnull().sum())

76
76
76
76


Therefore we can immediately fill those 76 rows with "NoGarage" value

In [18]:
mask = house_data["GarageType"].isnull()

for feature in ["GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond"]:
    house_data.loc[mask, feature] = house_data[feature][mask].fillna("NoGarage")

In [19]:
house_data["GarageType"] = house_data["GarageType"].fillna("NoGarage")

Now, the rest of missing values from Garage

In [20]:
print(house_data["GarageYrBlt"][house_data["GarageFinish"].isnull()].isnull().sum())
print(house_data["GarageYrBlt"][house_data["GarageQual"].isnull()].isnull().sum())
print(house_data["GarageYrBlt"][house_data["GarageCond"].isnull()].isnull().sum())

2
2
2


It seems that those 2 missing values correspond to the same row, let's print it:

In [21]:
garage_features = ["GarageType", "GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond", "GarageCars", "GarageArea"]
house_data[house_data["GarageFinish"].isnull()][garage_features]

,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,GarageCars,GarageArea
Id,,,,,,,
2127,Detchd,NaN,NaN,NaN,NaN,1.0,360.0
2577,Detchd,NaN,NaN,NaN,NaN,NaN,NaN


Since there is no obvious way of filling them in, I'm going to put mode in each of the features.

In case of GarageArea I'm going to replace the value by its geometrical mean (it's almost geometric mean since I'm using log(1+x) transformation and not log(x)) since it comes into the model by means of a logarithm.

In [22]:
for feature in ["GarageYrBlt", "GarageFinish", "GarageQual", "GarageCond", "GarageCars"]:
    house_data[feature] = house_data[feature].fillna(house_data[feature].mode()[0])

In [23]:
geometric_mean = np.exp(np.mean(np.log(house_data["GarageArea"] + 1))) - 1
house_data["GarageArea"] = house_data["GarageArea"].fillna(geometric_mean)

In [24]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage     227
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinSF1        1
BsmtFinType2     42
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
dtype: int64


Now we're only left with Basement features since we don't use LotFrontage in our models. From the outlook it seems that one row doesn't contain information about area and about 42 rows have categorical features missing. Let's check if this is the case: 

In [26]:
bsmt_features = ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2", 
                 "BsmtUnfSF", "TotalBsmtSF"]
mask = house_data["TotalBsmtSF"].isnull()
house_data[mask][bsmt_features]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
Id,,,,,,,,,
2121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So for this one row, no information about basement is available. I will therefore assume that it didn't have a basement at all.

In [27]:
for feature in ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2"]:
    house_data.loc[2121, feature] = "NoBsmt"
for feature in ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF"]:
    house_data.loc[2121, feature] = 0

In [28]:
mask = house_data["BsmtFinType1"].isnull()
house_data[mask][bsmt_features]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
Id,,,,,,,,,
1586,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1594,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1730,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1779,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1815,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1848,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1849,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1857,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0
1858,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0


Above means that all 41 of those houses, didn't have a basement.

In [29]:
# filling those 41 rows
mask = house_data["BsmtFinType1"].isnull()

for feature in ["BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType2"]:
    house_data.loc[mask, feature] = house_data[mask][feature].fillna("NoBsmt")

house_data["BsmtFinType1"] = house_data["BsmtFinType1"].fillna("NoBsmt")

In [30]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage     227
BsmtQual          2
BsmtCond          3
BsmtExposure      2
dtype: int64


In [32]:
mask = house_data["BsmtCond"].isnull()

print(house_data[mask][["BsmtQual", "BsmtCond", "BsmtExposure"]])

mask = house_data["BsmtQual"].isnull()

print(house_data[mask][["BsmtQual", "BsmtCond", "BsmtExposure"]])

mask = house_data["BsmtExposure"].isnull()

print(house_data[mask][["BsmtQual", "BsmtCond", "BsmtExposure"]])

     BsmtQual BsmtCond BsmtExposure
Id                                 
2041       Gd      NaN           Mn
2186       TA      NaN           No
2525       TA      NaN           Av
     BsmtQual BsmtCond BsmtExposure
Id                                 
2218      NaN       Fa           No
2219      NaN       TA           No
     BsmtQual BsmtCond BsmtExposure
Id                                 
1488       Gd       TA          NaN
2349       Gd       TA          NaN


We conclude they are single instances of missing variables. Therefore I will fill them with modes.

In [33]:
for feature in ["BsmtQual", "BsmtCond", "BsmtExposure"]:
    house_data[feature] = house_data[feature].fillna(house_data[feature].mode()[0])

In [34]:
# checking missing values in each column
missing_val_count_by_column = house_data.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    227
dtype: int64


In [35]:
# So we have a clear dataset. Let's save it to a file:
house_data.to_csv("data/test_preprocessed.csv")